In [1]:
import os
import pickle

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

In [2]:
def collect_results(case, dirname_results, dump_keys=None, voigt=False):

    if voigt:
        group, cell, suffix = case.split("/")
        group, cell = int(group[-1]), int(cell[-1])
    else:
        group, cell, suffix = None, None, case

    config_path = os.path.join(dirname_results, case)
    with open(os.path.join(config_path, "config_backup.pickle"), "rb") as f:
        config = pickle.load(f)

    m_index = config["runtime"]["m_index"]

    # n_genes = len(m_index)
    # n_organisms = config['runtime']['n_organisms']

    if dump_keys is None:
        dump_keys = ["dump", "best"]

    dump = {}
    for folder in dump_keys:
        dump[folder] = {}
        for key in "genes", "state", "status", "loss":
            filename = os.path.join(config_path, folder, key)
            if os.path.isfile(filename):
                dump[folder][key] = np.fromfile(filename)

    filename = os.path.join(config_path, "sol_best.csv")
    if os.path.isfile(filename):
        sol_best = pd.read_csv(filename, index_col=[0, 1]).iloc[:, -1]
    else:
        sol_best = None

    phenotype_best = {}

    for exp_cond_name in config["experimental_conditions"]:

        if exp_cond_name == "common":
            continue

        filename = os.path.join(
            config_path, "phenotype", f"phenotype_{exp_cond_name}.csv"
        )
        if os.path.isfile(filename):
            try:
                phenotype_best[exp_cond_name] = pd.read_csv(filename)
            except pd.errors.EmptyDataError as e:
                print(f"{filename} is empty")
                continue

    filename = os.path.join(config_path, "state_best.csv")
    state_best = pd.read_csv(filename, index_col=0)

    results = dict(
        trio=(group, cell, suffix),
        config=config,
        dump=dump,
        sol_best=sol_best,
        phenotype_best=phenotype_best,
        state_best=state_best,
    )

    return results

In [3]:
dirname_results = "Путь до mpi_scripts/results/"
case = "Номер расчета, например: 220427_110659"

results = collect_results(case, dirname_results, dump_keys=["best", "dump"])
results.keys()

dict_keys(['trio', 'config', 'dump', 'sol_best', 'phenotype_best', 'state_best'])

In [4]:
config = results["config"]

m_index = config["runtime"]["m_index"]

In [5]:
results["sol_best"]

ec_name  g_name
common   P_Na        0.944054
         g_K1        0.707945
1000     Na_i        8.086342
         K_i       130.300659
500      Na_i        8.686210
         K_i       131.620802
Name: 0, dtype: float64

In [6]:
dump_best_genes = pd.DataFrame(
    results["dump"]["best"]["genes"].reshape(-1, len(m_index)),
    columns=m_index
)

dump_best_loss = pd.Series(
    results["dump"]["best"]["loss"],
)

In [7]:
dump_best_genes

ec_name    common                 1000                    500            
g_name       P_Na      g_K1       Na_i         K_i       Na_i         K_i
0        0.963916  0.723945   8.271842  131.295175   9.332078  101.355641
1        0.963916  0.723945   8.225878  131.346839   9.279121  101.405465
2        1.472220  0.569920  10.044630  131.539728  10.812557  128.129793
3        1.590318  0.623589  10.063320  122.854020  12.078740  110.558151
4        0.944054  0.707945   8.086342  130.300659   8.686210  131.620802

In [8]:
dump_best_loss

0    26.296982
1    25.540216
2    26.287642
3    31.313943
4    30.753967
dtype: float64